In [85]:
# !pip3 install nltk

import re
import sys
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.sequence import pad_sequences

from nltk.tokenize import word_tokenize
import nltk

sys.path.append("/Users/wzq/Desktop/game")
from utils import load_glove_embeddings, sentence_to_embedding, save_zip_file

# jupyter配置
from IPython.display import display
pd.options.display.max_rows=10000 #Notebook 的一个cell的显示行数
pd.options.display.max_columns=100000#Notebook 的一个cell的显示列数
pd.set_option('display.max_colwidth', None)

In [86]:
## 数据处理
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')
# 类别均衡
duplicated_samples1 = train_data[train_data['rating'] == 1]
duplicated_samples2 = train_data[train_data['rating'] == 2]
train_data = pd.concat([train_data, duplicated_samples1, duplicated_samples2], axis=0, ignore_index=True)

In [87]:
# 计算用户特征
user_features = train_data.groupby('user_id').agg({
    'rating': ['mean', 'count', 'std', 'var'],
    'votes': ['mean', 'max'],
    'helpful_votes': ['mean', 'max']
}).reset_index()

user_features.columns = ['user_id', 'user_rating_mean', 'user_rating_count', 'user_rating_std', 'user_rating_var', 
                         'user_votes_mean', 'user_votes_max', 
                         'user_helpful_votes_mean', 'user_helpful_votes_max']
user_features['user_votes_helpful_rate'] = user_features['user_helpful_votes_mean'] / user_features['user_votes_mean']
user_features['user_helpful_rating'] = user_features['user_votes_helpful_rate'] * user_features['user_rating_mean']
user_features.head()

,user_id,user_rating_mean,user_rating_count,user_rating_std,user_rating_var,user_votes_mean,user_votes_max,user_helpful_votes_mean,user_helpful_votes_max,user_votes_helpful_rate,user_helpful_rating
0,0,4.572139,201,0.604996,0.366020,3.970149,18,3.059701,18,0.770677,3.523641
1,1,4.902222,225,0.326295,0.106468,2.342222,32,1.337778,27,0.571157,2.799941
2,2,3.564246,179,1.285204,1.651748,1.765363,25,0.614525,9,0.348101,1.240718
3,3,3.575758,495,1.231264,1.516010,7.888889,69,5.200000,47,0.659155,2.356978
4,4,4.083095,349,1.109766,1.231581,5.524355,34,4.449857,34,0.805498,3.288924


In [88]:
# 计算产品特征
product_features = train_data.groupby('product_id').agg({
    'rating': ['mean', 'count', 'std', 'var'],
    'votes': ['mean', 'max'],
    'helpful_votes': ['mean', 'max']
}).reset_index()
product_features.columns = ['product_id', 'product_rating_mean', 'product_rating_count', 'product_rating_std', 'product_rating_var',
                            'product_votes_mean', 'product_votes_max', 'product_helpful_votes_mean', 'product_helpful_votes_max']
product_features['product_votes_helpful_rate'] = product_features['product_helpful_votes_mean'] / product_features['product_votes_mean']
product_features['product_helpful_rating'] = product_features['product_votes_helpful_rate'] * product_features['product_rating_mean']

product_features.head()
glove_embeddings = load_glove_embeddings('/Users/wzq/Desktop/coding/glove.6B/glove.6B.50d.txt')


In [89]:
## 数据处理
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')
# 生成用户和产品特征
train_data = train_data.merge(user_features, on='user_id').merge(product_features, on='product_id')
test_data = test_data.merge(user_features, on='user_id', how='left').merge(product_features, on='product_id', how='left')



# 分词并加载词嵌入
def sentence_to_embedding(sentence, embeddings_index, enable_mean=False, embedding_dim=50):
    words = word_tokenize(sentence.lower())  # 分词，并转化为小写
    embeddings = []
    for word in words:
        if word in embeddings_index:
            embeddings.append(embeddings_index[word])
        else:
            embeddings.append(np.zeros(embedding_dim))  # 若词不在 GloVe 中，则用零向量替代
    if enable_mean:
        embeddings = np.mean(embeddings, axis=0)
    return embeddings

# 文本特征
# 假设路径为 'glove.6B.100d.txt'
glove_embeddings = load_glove_embeddings('/Users/wzq/Desktop/coding/glove.6B/glove.6B.50d.txt')
train_sentence_embeddings = train_data['product_name'].apply(lambda x: sentence_to_embedding(x, glove_embeddings, enable_mean=True, embedding_dim=50))
train_sentence_embeddings = pd.DataFrame(train_sentence_embeddings.tolist())
test_sentence_embeddings = test_data['product_name'].apply(lambda x: sentence_to_embedding(x, glove_embeddings, enable_mean=True, embedding_dim=50))
test_sentence_embeddings = pd.DataFrame(test_sentence_embeddings.tolist())
# display(test_sentence_embeddings)

# embeddings_padded = pad_sequences(sentence_embedding_mean.tolist(), maxlen=14 * 50, dtype='float32', padding='post', truncating='post')
for i in range(50):
    cn = 'product_name_embeddings_' + str(i)
    train_data[cn] = train_sentence_embeddings.iloc[:, i]
    test_data[cn] = test_sentence_embeddings.iloc[:, i]

# 填充缺失值
train_data.fillna(0, inplace=True)
test_data.fillna(0, inplace=True)  # 或者使用其他合适的填充方法

# 保存训练测试数据
train_data.to_csv('./data/train_processed_bls_cls.csv')
test_data.to_csv('./data/test_processed_bls_cls.csv')
# train_data.to_csv('./data/train_processed.csv')
# test_data.to_csv('./data/test_processed.csv')
# train_data.to_csv('./data/train_processed_product_name.csv')
# test_data.to_csv('./data/test_processed_product_name.csv')
display(train_data)
display(test_data)

,user_id,product_id,product_name,rating,votes,helpful_votes,ID,user_rating_mean,user_rating_count,user_rating_std,user_rating_var,user_votes_mean,user_votes_max,user_helpful_votes_mean,user_helpful_votes_max,user_votes_helpful_rate,user_helpful_rating,product_rating_mean,product_rating_count,product_rating_std,product_rating_var,product_votes_mean,product_votes_max,product_helpful_votes_mean,product_helpful_votes_max,product_votes_helpful_rate,product_helpful_rating,product_name_embeddings_0,product_name_embeddings_1,product_name_embeddings_2,product_name_embeddings_3,product_name_embeddings_4,product_name_embeddings_5,product_name_embeddings_6,product_name_embeddings_7,product_name_embeddings_8,product_name_embeddings_9,product_name_embeddings_10,product_name_embeddings_11,product_name_embeddings_12,product_name_embeddings_13,product_name_embeddings_14,product_name_embeddings_15,product_name_embeddings_16,product_name_embeddings_17,product_name_embeddings_18,product_name_embeddings_19,product_name_embeddings_20,product_name_embeddings_21,product_name_embeddings_22,product_name_embeddings_23,product_name_embeddings_24,product_name_embeddings_25,product_name_embeddings_26,product_name_embeddings_27,product_name_embeddings_28,product_name_embeddings_29,product_name_embeddings_30,product_name_embeddings_31,product_name_embeddings_32,product_name_embeddings_33,product_name_embeddings_34,product_name_embeddings_35,product_name_embeddings_36,product_name_embeddings_37,product_name_embeddings_38,product_name_embeddings_39,product_name_embeddings_40,product_name_embeddings_41,product_name_embeddings_42,product_name_embeddings_43,product_name_embeddings_44,product_name_embeddings_45,product_name_embeddings_46,product_name_embeddings_47,product_name_embeddings_48,product_name_embeddings_49
0,1813,154533,Beautiful Thing,5,10,8,0,4.001621,617,1.274754,1.624997,7.951378,88,5.367909,70,0.675092,2.701461,4.555556,9,0.527046,0.277778,4.444444,10,4.0,8,0.90,4.10,0.268777,0.704065,-0.704850,-0.046943,0.968450,0.016998,-0.354835,-0.106115,-0.204095,0.584430,-0.540680,0.216230,-0.051420,0.228735,0.667465,0.111256,0.746360,0.454310,0.051473,-0.558030,-0.694385,0.790550,-0.134327,0.267640,1.188850,-1.233670,-1.544600,0.812890,1.037625,-0.809220,2.534800,0.014870,0.088995,-0.118358,0.005690,0.203015,-0.046273,0.437550,-0.247170,-0.671570,-0.378545,0.126550,0.087245,0.062750,0.059166,0.266091,0.013202,-0.767775,0.244955,0.535740
1,1916,154533,Beautiful Thing,4,2,2,681987,4.241384,121089,1.282646,1.645180,7.666477,681,5.275615,657,0.688141,2.918670,4.555556,9,0.527046,0.277778,4.444444,10,4.0,8,0.90,4.10,0.268777,0.704065,-0.704850,-0.046943,0.968450,0.016998,-0.354835,-0.106115,-0.204095,0.584430,-0.540680,0.216230,-0.051420,0.228735,0.667465,0.111256,0.746360,0.454310,0.051473,-0.558030,-0.694385,0.790550,-0.134327,0.267640,1.188850,-1.233670,-1.544600,0.812890,1.037625,-0.809220,2.534800,0.014870,0.088995,-0.118358,0.005690,0.203015,-0.046273,0.437550,-0.247170,-0.671570,-0.378545,0.126550,0.087245,0.062750,0.059166,0.266091,0.013202,-0.767775,0.244955,0.535740
2,506,154533,Beautiful Thing,5,6,6,288065,4.763975,483,0.599262,0.359114,2.347826,20,1.685300,16,0.717813,3.419644,4.555556,9,0.527046,0.277778,4.444444,10,4.0,8,0.90,4.10,0.268777,0.704065,-0.704850,-0.046943,0.968450,0.016998,-0.354835,-0.106115,-0.204095,0.584430,-0.540680,0.216230,-0.051420,0.228735,0.667465,0.111256,0.746360,0.454310,0.051473,-0.558030,-0.694385,0.790550,-0.134327,0.267640,1.188850,-1.233670,-1.544600,0.812890,1.037625,-0.809220,2.534800,0.014870,0.088995,-0.118358,0.005690,0.203015,-0.046273,0.437550,-0.247170,-0.671570,-0.378545,0.126550,0.087245,0.062750,0.059166,0.266091,0.013202,-0.767775,0.244955,0.535740
3,923,154533,Beautiful Thing,5,9,8,365036,4.261745,149,0.910865,0.829675,11.154362,94,10.134228,92,0.908544,3.871982,4.555556,9,0.527046,0.277778,4.444444,10,4.0,8,0.90,4.10,0.268777,0.704065,-0.704850,-0.046943,0.968450,0.016998,-0.354835,-0.106115,-0.204095,0.5

,ID,user_id,product_id,product_name,user_rating_mean,user_rating_count,user_rating_std,user_rating_var,user_votes_mean,user_votes_max,user_helpful_votes_mean,user_helpful_votes_max,user_votes_helpful_rate,user_helpful_rating,product_rating_mean,product_rating_count,product_rating_std,product_rating_var,product_votes_mean,product_votes_max,product_helpful_votes_mean,product_helpful_votes_max,product_votes_helpful_rate,product_helpful_rating,product_name_embeddings_0,product_name_embeddings_1,product_name_embeddings_2,product_name_embeddings_3,product_name_embeddings_4,product_name_embeddings_5,product_name_embeddings_6,product_name_embeddings_7,product_name_embeddings_8,product_name_embeddings_9,product_name_embeddings_10,product_name_embeddings_11,product_name_embeddings_12,product_name_embeddings_13,product_name_embeddings_14,product_name_embeddings_15,product_name_embeddings_16,product_name_embeddings_17,product_name_embeddings_18,product_name_embeddings_19,product_name_embeddings_20,product_name_embeddings_21,product_name_embeddings_22,product_name_embeddings_23,product_name_embeddings_24,product_name_embeddings_25,product_name_embeddings_26,product_name_embeddings_27,product_name_embeddings_28,product_name_embeddings_29,product_name_embeddings_30,product_name_embeddings_31,product_name_embeddings_32,product_name_embeddings_33,product_name_embeddings_34,product_name_embeddings_35,product_name_embeddings_36,product_name_embeddings_37,product_name_embeddings_38,product_name_embeddings_39,product_name_embeddings_40,product_name_embeddings_41,product_name_embeddings_42,product_name_embeddings_43,product_name_embeddings_44,product_name_embeddings_45,product_name_embeddings_46,product_name_embeddings_47,product_name_embeddings_48,product_name_embeddings_49
0,0,1916,185507,Maria [Australia],4.241384,121089,1.282646,1.645180,7.666477,681,5.275615,657,0.688141,2.918670,5.000000,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,-0.127962,0.760284,0.033650,0.293735,0.133326,0.157034,-0.091510,-0.019397,-0.297530,-0.227621,0.257265,0.350088,-0.270132,-0.506997,0.450198,-0.155560,0.073660,-0.186825,-0.231528,0.273275,-0.368023,0.083898,0.445597,0.483992,0.582603,-0.707140,0.039945,-0.421305,-0.350972,-0.245525,2.197750,-0.142304,0.020252,-0.104307,-0.298925,-0.397750,0.493244,0.058490,0.206745,-0.356597,0.426220,-0.404720,0.360432,-0.575941,-0.168985,0.620425,-0.249920,-0.631971,0.307535,0.412900
1,1,1759,143430,"The Mitchells: Five for Victory (Van Stockum, Hilda, ""Mitchells"" Series.)",3.738132,653,1.733034,3.003406,6.539051,91,3.849923,42,0.588759,2.200858,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,-0.061768,0.290024,-0.100216,-0.021020,0.109157,0.226238,-0.398371,-0.341678,-0.243907,-0.285389,-0.098447,0.164986,-0.535192,-0.195681,0.342699,-0.021038,0.222990,-0.066948,-0.486469,-0.145144,-0.091811,0.224055,0.320318,0.096743,-0.009046,-0.917202,-0.397798,-0.093497,-0.000371,-0.312151,2.431537,-0.030600,-0.278046,0.125526,0.173048,-0.036254,0.243203,-0.091143,0.288496,-0.093532,0.074945,0.119532,0.117767,-0.343035,-0.001427,0.294062,-0.017395,-0.351465,0.088295,0.139058
2,2,1980,155861,Superman Cartoons of Max & Dav,3.626374,546,1.503379,2.260147,6.304029,52,3.478022,41,0.551714,2.000722,4.714286,7.0,0.487950,0.238095,12.285714,57.0,11.428571,56.0,0.930233,4.385382,-0.304495,0.167180,-0.113720,-0.001227,-0.048765,0.655275,-0.328147,-0.928585,-0.137822,-0.370920,0.034489,0.528562,-0.486520,-0.436582,0.431873,-0.125909,-0.020044,0.601835,-0.405915,0.094026,0.217900,0.130097,0.152542,0.250303,0.303188,-0.670387,-0.595923,-0.252234,-0.213731,-0.194479,1.248129,0.104120,0.056142,-0.297540,-0.362604,0.587309,0.054943,-0.398184,-0.168702,-0.112553,0.283088,0.202617,-0.445486,-0.477901,0.101660,0.550085,0.048121,-0.351010,-0.101229,0.035824
3,3,1502,71535,The Rainbow Fish Anniversary Edition,4.052681,31131,1.377479,1.897450,4.240982,380,1.761556,147,0.415365,1.683342,3.125000,8.0,1.807722,3.267

In [71]:
train_data['rating'].value_counts()
train_data.shape

(745889, 7)

In [84]:
duplicated_samples1 = train_data[train_data['rating'] == 1]
duplicated_samples2 = train_data[train_data['rating'] == 2]
a = pd.concat([train_data, duplicated_samples1, duplicated_samples2], axis=0, ignore_index=True)
a['rating'].value_counts()

rating
5    416231
4    185193
3     79870
2     70892
1     58298
Name: count, dtype: int64

In [105]:
a = pd.read_csv('./data/submit_example_A202410161035.csv')
# display(a['rating'].value_counts())
# a[a['rating'] > 5]['rating'] = 5
a.loc[a['rating'] > 5, 'rating'] = 5
# save_zip_file(a)
a['rating'] = a['rating'].round()
cnt = a['rating'].value_counts()
type(cnt)
for i in cnt:
    print(i, i / sum(cnt))

108373 0.48477542238305904
74501 0.33325877979718455
39342 0.1759851131499018
1335 0.005971738245516723
2 8.946424337852769e-06


In [114]:
l = [416231,185193,79870,35446,29149]
l = [101075, 76738, 41891, 3153, 642]
for i in l:
    print(i, i / sum(l))

101075 0.4522391599067557
76738 0.3433482923860957
41891 0.1874326059624428
3153 0.014107445670897856
642 0.0028724960738079363


In [9]:
a = np.array([[7.62842181e-01, 4.90758275e-03, 1.37024614e-01],
        [2.67379679e-03, 1.37931034e-02, 3.15333071e-03],
        [8.28423978e-01, 1.00000000e+00, 1.31421749e-01],
        [2.67379679e-03, 1.37931034e-02, 3.15333071e-03],
        [9.31983403e-01, 4.23190107e-03, 3.72131582e-02],
        [2.67379679e-03, 1.37931034e-02, 3.15333071e-03],
        [7.23851417e-01, 1.13799020e-03, 4.92168973e-02],
        [5.34759358e-04, 3.44827586e-03, 3.94166338e-04],
        [7.23851417e-01, 1.13799020e-03, 4.92168973e-02],
        [5.34759358e-04, 6.89655172e-03, 7.88332676e-04],
        [7.23851417e-01, 1.13799020e-03, 4.92168973e-02],
        [8.02139037e-04, 1.03448276e-02, 1.18249901e-03]])

In [118]:
t = [416231,185193,79870,70892,58298]
for i in t:
    print(i, i / sum(t))
sum(t)

416231 0.5135585650056016
185193 0.22849679944329562
79870 0.09854605396281728
70892 0.08746872239303922
58298 0.0719298591952463


810484

In [4]:
field_dims = [5, 10, 15]  # 三个字段，每个字段有不同数量的类别
offsets = np.array((0, *np.cumsum(field_dims)[:-1]))
offsets

array([ 0,  5, 15, 30])

In [51]:
from utils import save_zip_file


re = pd.read_csv('data/submit_example_A202410152136.csv')
re['rating'] = re['rating'].round()
save_zip_file(re)

In [ ]:
## 文本分析
df_unique_by_columns = train_data.drop_duplicates(subset=['product_name', 'product_id'])[['product_name', 'product_id']]
df_unique_by_columns.head(10000).sort_values('product_name')
product_name = df_unique_by_columns['product_name']
# display(product_name)
# 分析
# tokenized_data = [word_tokenize(sentence.lower()) for sentence in product_name]
# tokenized_data
# model = KeyedVectors.load_word2vec_format('path/to/glove.6B.100d.txt', binary=False)
# lengths = [len(s) for s in tokenized_data]
# max_len = int(np.percentile(lengths, 90)) #  lengths 列表中计算出覆盖 90% 样本的最大长度。 14


# def load_glove_embeddings(file_path):
#     embeddings_index = {}
#     with open(file_path, 'r', encoding='utf-8') as f:
#         for line in f:
#             values = line.split()
#             word = values[0]
#             embedding_vector = np.asarray(values[1:], dtype='float32')
#             embeddings_index[word] = embedding_vector
#     return embeddings_index

# # 分词并加载词嵌入
# def sentence_to_embedding(sentence, embeddings_index, embedding_dim=100):
#     words = word_tokenize(sentence.lower())  # 分词，并转化为小写
#     embeddings = []
#     for word in words:
#         if word in embeddings_index:
#             embeddings.extend(embeddings_index[word])
#         else:
#             embeddings.extend(np.zeros(embedding_dim))  # 若词不在 GloVe 中，则用零向量替代
#     return embeddings

# 为每一句话生成嵌入矩阵
tmp = pd.DataFrame()
tmp['product_name'] = train_data['product_name'].iloc[:10]
tmp['product_name_embeddings'] = tmp['product_name'].apply(lambda x: sentence_to_embedding(x, glove_embeddings))

# display(tmp[['product_name','product_name_embeddings']].head())
embeddings_padded = pad_sequences(tmp['product_name_embeddings'].tolist(), maxlen=14 * 100, dtype='float32', padding='post', truncating='post')
for i in range(14):
    cn = 'product_name_embeddings_' + str(i)
    tmp[cn] = embeddings_padded[:, i]
display(tmp.head())


# plt.figure(figsize=(10,6))
# plt.hist(lengths, bins=range(1, max(lengths) + 2), alpha=0.7)
# plt.title('Sentence Length Distribution')
# plt.xlabel('Length of Sentences (Number of Words)')
# plt.ylabel('Frequency')
# plt.xticks(range(1, max(lengths) + 1), rotation=45)
# plt.grid(axis='y', alpha=0.75)
# plt.show()